In [ ]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from xbbg import blp # Bloomberg Python API
from datetime import date, timedelta

In this notebook, we seek to compare the differences between a non-linear (logistic) scaling function, a linear scaling function, and a constant linear function in reversion-based arrival price trading. This is not investment advice.

In [ ]:
# select one stock at random from each of four indexes: S&P 100 (OEX), S&P 500 (SPX), S&P 400 (MID), & S&P 600 (SML)
index = ["OEX Index","SPX Index","MID Index","SML Index"]
test_sym = blp.bds(tickers = index, flds = "INDX_MEMBERS").reset_index().rename(columns = {'index':'index_id'})
test_sym = test_sym.groupby('index_id')[['index_id','member_ticker_and_exchange_code']].apply(lambda x: x.sample(1)).reset_index(drop=True)['member_ticker_and_exchange_code'].tolist()
test_sym = [sym[:-3] + " US Equity" for sym in test_sym] #modify symbology to match Bloomberg convention

for sym in test_sym:
    print(sym)

In [ ]:
# select 30 full trading days in Q1 2025
def get_days(start_date, end_date):
    dates = []
    q1_holidays_us = [date(2025,1,1),
                      date(2025,1,20),
                      date(2025,2,17)] # holidays and half-days

    current_date = start_date
    while current_date <= end_date:
        if current_date not in q1_holidays_us and current_date.weekday() < 5: # Monday is 0, Sunday is 6
            dates.append(current_date)
        current_date += timedelta(days=1)
    return dates

start_date = date(2025, 1, 1)
end_date = date(2025, 3, 31)
day_list = get_days(start_date, end_date)
test_days: list = random.sample(day_list, 30)

In [ ]:
# Compare Fixed POV, Clamped Linear, & 4PL outcomes
outputs = [[] for _ in range(8)]

for sym in test_sym:
    for dt in test_days: 
        # trade, volume, and volatility data from bbg
        ohlc = blp.bdib(ticker = sym, dt = dt).reset_index(drop=True)
        ohlc = ohlc[ohlc.index < 390] 
        twentyd_vol = blp.bdh(tickers = sym, flds = ["VOLATILITY_20D"], start_date = dt, end_date = dt).reset_index(drop=True)
        twentyd_volm = blp.bds(tickers = sym, flds=["VOLUME_AVG_20D"]).reset_index(drop=True)
        twentyd_vol.columns = twentyd_vol.columns.droplevel()
        ohlc.columns = ohlc.columns.droplevel()

        # Establish  daily volatility, and 1-minute VWAP
        stddev_1d = twentyd_vol.loc[0, 'VOLATILITY_20D']/math.sqrt(252)/100
        ohlc['VWAP_1m'] = (ohlc['value'] / ohlc['volume'])

        # Set volume, volatility, and benchmark parameters
        benchmark = ohlc.loc[0, 'open']
        vol_min = 0.05 # set to 5% min vol
        vol_max = 0.15 # set to 15% max vol
        vol_mean = (vol_min + vol_max) / 2
        px_max = benchmark * (1 + 1.5 * stddev_1d) # tuned to 3 std dev
        px_min = benchmark * (1 - 1.5 * stddev_1d) # tuned to 3 std dev

        # Linear function slope and Y-intercept
        m = (vol_max - vol_min) / (px_min - px_max)
        b = vol_mean - (m * benchmark) 
        # Linear and logistic functions
        ohlc['pRate Flat'] = round(vol_mean,3) # part rate rounded to 0.000
        ohlc['pRate Lin'] = round(np.minimum(vol_max, np.maximum(vol_min, (m * ohlc['open']) + b)),3) # part rate rounded to 0.000
        ohlc['pRate Log'] = round(vol_mean / (1 + np.exp(-(4 * m / vol_mean) * (ohlc['open'] - benchmark))) + vol_min,3) # part rate rounded to 0.000

        # Set the order and fill qty
        flat_leaves = round(twentyd_volm.loc[0, 'value'] * 0.05,0) # order qty set to 5 percent of the 20-day ADV
        lin_leaves = round(twentyd_volm.loc[0, 'value'] * 0.05,0) # order qty set to 5 percent of the 20-day ADV
        log_leaves = round(twentyd_volm.loc[0, 'value'] * 0.05,0) # order qty set to 5 percent of the 20-day ADV
        flat_filled_qty = 0
        lin_filled_qty = 0
        log_filled_qty = 0

        for i in ohlc.index:
            ohlc.loc[i,'pRate Flat fills'] = min(lin_leaves, round(ohlc.loc[i,'pRate Flat'] * ohlc.loc[i,'volume'],0))
            ohlc.loc[i,'pRate Lin fills'] = min(lin_leaves, round(ohlc.loc[i,'pRate Lin'] * ohlc.loc[i,'volume'],0))
            ohlc.loc[i,'pRate Log fills'] = min(log_leaves, round(ohlc.loc[i,'pRate Log'] * ohlc.loc[i,'volume'],0))

            flat_filled_qty += ohlc.loc[i,'pRate Flat fills']
            lin_filled_qty += ohlc.loc[i,'pRate Lin fills']
            log_filled_qty += ohlc.loc[i,'pRate Log fills']

            flat_leaves -= ohlc.loc[i,'pRate Flat fills']
            lin_leaves -= ohlc.loc[i,'pRate Lin fills']
            log_leaves -= ohlc.loc[i,'pRate Log fills']

            ohlc.loc[i,'pRate Flat filled'] = flat_filled_qty
            ohlc.loc[i,'pRate Lin filled'] = lin_filled_qty
            ohlc.loc[i,'pRate Log filled'] = log_filled_qty

            ohlc.loc[i,'flat_leaves'] = max(0,flat_leaves)
            ohlc.loc[i,'lin_leaves'] = max(0,lin_leaves)
            ohlc.loc[i,'Log_leaves'] = max(0,log_leaves)

        outputs[0].append(sym)
        outputs[1].append(dt)
        outputs[2].append((ohlc['pRate Flat fills'] * ohlc['VWAP_1m']).sum() / flat_filled_qty)
        outputs[3].append((ohlc['pRate Lin fills'] * ohlc['VWAP_1m']).sum() / lin_filled_qty)
        outputs[4].append((ohlc['pRate Log fills'] * ohlc['VWAP_1m']).sum() / log_filled_qty)

        outputs[5].append(flat_filled_qty)
        outputs[6].append(lin_filled_qty)
        outputs[7].append(log_filled_qty)

    d = {"symbol":outputs[0],"date":outputs[1],"flat":outputs[2],"linear":outputs[3],"log":outputs[4],"flat qty":outputs[5],"lin_qty":outputs[6],"log qty":outputs[7]}
    analysis_df = pd.DataFrame(data=d)

    # Relative t-tests and p-values
    t_flt_lin, p_flt_lin = stats.ttest_rel(analysis_df['flat'], analysis_df['linear'], alternative='greater')
    t_lin_log, p_lin_log = stats.ttest_rel(analysis_df['linear'], analysis_df['log'], alternative='greater')

    # Statistical Analysis, Fixed POV vs Clamped Linear
    print("Paired One-Tailed t-Test:", sym, "\n")
    print("Fixed POV [n, Mean, Variance]", analysis_df['flat'].count(), round(analysis_df['flat'].mean(), 4), round(analysis_df['flat'].var(), 4))
    print("Clamped Linear [n, Mean, Variance]", analysis_df['linear'].count(), round(analysis_df['linear'].mean(), 4), round(analysis_df['linear'].var(), 4))
    print("t_statistic", round(t_flt_lin, 8))
    print("p-value", (p_flt_lin), "\n")

    # Statistical Analysis, Clamped Linear vs 4PL
    print("Clamped Linear [n, Mean, Variance]", analysis_df['linear'].count(), round(analysis_df['linear'].mean(), 4), round(analysis_df['linear'].var(), 4))
    print("4PL [n, Mean, Variance]", analysis_df['log'].count(), round(analysis_df['log'].mean(), 4), round(analysis_df['log'].var(), 4))
    print("t_statistic", round(t_lin_log, 8))
    print("p-value", (p_lin_log))

In [ ]:
# Visual comparison of Fixed POV, Clamped Linear, & 4PL
font_title = {'color':'black', 'size':14}
font = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 12}

flat = ohlc['pRate Flat']
lin = ohlc['pRate Lin']
log = ohlc['pRate Log']

fig, ax = plt.subplots(nrows=1,ncols=1)

ax.set_xlim(0,ohlc['open'].count())
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.plot(lin, label ='Clamped Linear',color='#0ead69',lw=1)
ax.plot(log, label ='4PL',color='#ff1654',dashes=[2,1],lw=1)
ax.plot(flat, label ='Fixed',color='#000000',dashes=[2,2],lw=1)
ax.set_ylim(.05,.15)
ax.set_xlim(0,ohlc['open'].count())

plt.rc('font', **font)
plt.title(str(dt) +" | "+ str(sym),fontdict=font_title)
plt.legend(frameon=False)
plt.ylabel("Percent of Volume")
plt.yticks([0.05, 0.1,.15])
plt.xticks([])

In [ ]:
# Visualization of Stock Price
y_max = ohlc.loc[0,'open'] + round(max(abs(ohlc['open'].max()-ohlc.loc[0,'open']),abs(ohlc.loc[0,'open'] - ohlc['open'].min())), 2)
y_min = ohlc.loc[0,'open'] - round(max(abs(ohlc['open'].max()-ohlc.loc[0,'open']),abs(ohlc.loc[0,'open'] - ohlc['open'].min())), 2)
fig, ax = plt.subplots(nrows=1,ncols=1)

ax.plot(ohlc['open'], label ='Price',color='#00aaff',lw=1)
ax.set_ylim(y_min, y_max)
ax.set_xlim(0,ohlc['open'].count())
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.rc('font', **font)
plt.title(str(dt) + " | " + str(sym),fontdict=font_title)
plt.ylabel("Price")
plt.grid()
plt.yticks([y_min, ohlc.loc[0,'open'],y_max])
plt.xticks([])